In [1]:
import onnx

model = onnx.load("transpose_net_241217.onnx")
for input_tensor in model.graph.input:
    print(f"Input Name: {input_tensor.name}, Shape: {input_tensor.type.tensor_type.shape.dim}")
for output_tensor in model.graph.output:
    print(f"Output Name: {output_tensor.name}, Shape: {output_tensor.type.tensor_type.shape.dim}")


Input Name: acc, Shape: [dim_param: "batch_size"
, dim_value: 18
]
Input Name: ori, Shape: [dim_param: "batch_size"
, dim_value: 54
]
Output Name: pose, Shape: [dim_param: "batch_size"
]
Output Name: tran, Shape: [dim_param: "batch_size"
]


In [3]:
import onnxruntime as ort
import numpy as np

# ONNX 모델 실행 준비
session = ort.InferenceSession("transpose_net_241217.onnx")

# 모델 입력 정보 확인
input_names = [inp.name for inp in session.get_inputs()]
output_names = [out.name for out in session.get_outputs()]
print(f"Input Names: {input_names}")
print(f"Output Names: {output_names}")

# 더미 입력 생성 (입력 크기에 맞춰 설정)
dummy_input = {
    "acc": np.random.randn(1, 18).astype(np.float32),
    "ori": np.random.randn(1, 54).astype(np.float32),
}

print(dummy_input)

# 실행 및 결과 출력
results = session.run(output_names, dummy_input)
for name, result in zip(output_names, results):
    print(f"Output - {name}: shape {result.shape}")

print(results)

Input Names: ['acc', 'ori']
Output Names: ['pose', 'tran']
{'acc': array([[ 0.50215614,  1.4753242 ,  0.653032  ,  0.9981128 ,  0.24920927,
        -1.0688585 , -1.4402975 ,  0.8658008 , -0.73913854, -0.21190351,
        -0.68251497, -0.2522456 ,  1.7773371 ,  0.7503529 ,  0.12864469,
         1.6468254 , -0.10820701, -0.18495123]], dtype=float32), 'ori': array([[-3.3806923e-01, -7.6558635e-02,  7.2830474e-01, -1.0559250e+00,
         9.4892710e-01, -1.3690926e-01,  1.5288116e-03, -1.8442574e-01,
         4.9888641e-01, -1.5487947e+00, -8.5241622e-01, -1.9409300e+00,
         6.6566712e-01, -7.2122931e-01, -8.6438233e-01,  2.5761659e+00,
         1.2458873e-01,  4.1176549e-01,  6.2004465e-01,  1.7553265e+00,
        -1.9800559e+00, -1.0109950e+00, -1.0835649e+00,  1.0015974e+00,
        -1.2207077e+00,  6.7213053e-01, -1.7201130e-01,  1.6866050e+00,
         8.3295447e-01,  5.3914666e-01,  5.4955256e-01, -8.5968137e-01,
         1.0170464e-01,  7.9847389e-01, -3.2259640e-01, -3.0478885

In [6]:
import torch
import json

# acc.pt 데이터 로드
acc_tensor = torch.load("C:/Users/sage/SynologyDrive/dev/transpose_livestream/client/acc_240521.pt")
acc_data = acc_tensor.numpy().tolist()

# ori.pt 데이터 로드
ori_tensor = torch.load("C:/Users/sage/SynologyDrive/dev/transpose_livestream/client/ori_240521.pt")
ori_data = ori_tensor.numpy().tolist()

# JSON 파일로 저장
with open("acc_240521.json", "w") as f:
    json.dump(acc_data, f)

with open("ori_240521.json", "w") as f:
    json.dump(ori_data, f)


C:\Users\sage\AppData\Local\Temp\ipykernel_51036\3248794920.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  acc_tensor = torch.load("C:/Users/sage/SynologyDrive/dev/tran

In [21]:
from utils import normalize_and_concat
from net import TransPoseNet
from articulate.math import *
# python 11에서 발생하는 버그 수정
import inspect

if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inertial_poser = TransPoseNet(num_past_frame=20, num_future_frame=5).to(device)

data_nn = normalize_and_concat(acc_tensor[0].view(1,18), ori_tensor[0].view(1,54)).to(device)
pose, tran = inertial_poser.forward_online(data_nn)
pose = rotation_matrix_to_axis_angle(pose.view(1, 216)).view(72)

pose

tensor([-0.0053,  0.0065,  0.0008,  0.0458,  0.0275,  0.0646,  0.0119, -0.0346,
        -0.0230,  0.0552,  0.0394,  0.0051, -0.0051, -0.0374, -0.0382,  0.0256,
         0.0056,  0.0099, -0.0494, -0.0521,  0.0056,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0581, -0.0029, -0.0158,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, -0.0109,  0.0597, -0.0200, -0.0892,
         0.0721,  0.0911, -0.0888, -0.0605, -0.0685, -0.0498, -0.0510,  0.0668,
        -0.2088,  0.0436, -0.3153, -0.2370,  0.0330,  0.3202,  0.2264, -0.0701,
         0.1556,  0.2939,  0.0506, -0.1717,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])